# Temporal Patterns Analysis

This notebook analyzes Citi Bike usage patterns around Columbia University focusing on temporal variations (hourly, daily, seasonal).

**Research Question 1:** How does Citi Bike usage vary near Columbia University by season, weekday, and time of day?

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

## 1. Data Loading & Feature Engineering

Load the filtered dataset and create temporal features for analysis.

In [ ]:
# Load the filtered data
data_path = os.path.join('..', 'data', 'columbia_filtered_citibike.csv')
df = pd.read_csv(data_path, parse_dates=['started_at', 'ended_at'])

print(f"Loaded {len(df):,} trips")
print(f"Date range: {df['started_at'].min()} to {df['started_at'].max()}")

In [ ]:
# Calculate trip duration in minutes
df['trip_duration_minutes'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60

# Filter outliers: remove trips < 1 minute or > 180 minutes (3 hours)
print(f"Trips before filtering: {len(df):,}")
df = df[(df['trip_duration_minutes'] >= 1) & (df['trip_duration_minutes'] <= 180)]
print(f"Trips after filtering: {len(df):,}")
print(f"Removed {529908 - len(df):,} outlier trips ({((529908 - len(df)) / 529908 * 100):.2f}%)")

In [ ]:
# Extract temporal features
df['hour_of_day'] = df['started_at'].dt.hour
df['day_of_week'] = df['started_at'].dt.dayofweek  # 0=Monday, 6=Sunday
df['day_name'] = df['started_at'].dt.day_name()
df['month'] = df['started_at'].dt.month
df['month_name'] = df['started_at'].dt.strftime('%Y-%m')
df['date'] = df['started_at'].dt.date

# Categorical features
df['is_weekend'] = df['day_of_week'] >= 5

# Season mapping (Northern Hemisphere)
def get_season(month):
	if month in [12, 1, 2]:
		return 'Winter'
	elif month in [3, 4, 5]:
		return 'Spring'
	elif month in [6, 7, 8]:
		return 'Summer'
	else:
		return 'Fall'

df['season'] = df['month'].apply(get_season)

# Time period categorization
def get_time_period(hour):
	if 6 <= hour < 10:
		return 'Morning Rush'
	elif 10 <= hour < 16:
		return 'Midday'
	elif 16 <= hour < 20:
		return 'Evening Rush'
	else:
		return 'Night'

df['time_period'] = df['hour_of_day'].apply(get_time_period)

print("\nFeatures created:")
print(df[['started_at', 'hour_of_day', 'day_name', 'is_weekend', 'season', 'time_period']].head())

---

## 2. Basic Usage Statistics

Overview of trip characteristics and user types.

In [ ]:
# Calculate total days in dataset
total_days = (df['started_at'].max() - df['started_at'].min()).days + 1

print("=== OVERALL STATISTICS ===")
print(f"Total trips: {len(df):,}")
print(f"Date range: {df['started_at'].min().date()} to {df['started_at'].max().date()}")
print(f"Total days: {total_days:,}")
print(f"Average trips per day: {len(df) / total_days:.1f}")
print(f"\n=== USER TYPE DISTRIBUTION ===")
print(df['member_casual'].value_counts())
print(f"\nMember percentage: {(df['member_casual'] == 'member').sum() / len(df) * 100:.1f}%")
print(f"\n=== BIKE TYPE DISTRIBUTION ===")
print(df['rideable_type'].value_counts())
print(f"\nElectric bike percentage: {(df['rideable_type'] == 'electric_bike').sum() / len(df) * 100:.1f}%")
print(f"\n=== TRIP DURATION STATISTICS (minutes) ===")
print(f"Median: {df['trip_duration_minutes'].median():.1f}")
print(f"25th percentile: {df['trip_duration_minutes'].quantile(0.25):.1f}")
print(f"75th percentile: {df['trip_duration_minutes'].quantile(0.75):.1f}")
print(f"Mean: {df['trip_duration_minutes'].mean():.1f}")
print(f"\nNote: Using median for analysis due to lognormal distribution of trip durations")

In [ ]:
# Visualize user type and bike type distributions
fig = make_subplots(
	rows=1, cols=2,
	subplot_titles=('User Type Distribution', 'Bike Type Distribution'),
	specs=[[{'type': 'pie'}, {'type': 'pie'}]]
)

# User type pie chart
user_counts = df['member_casual'].value_counts()
fig.add_trace(
	go.Pie(labels=user_counts.index, values=user_counts.values, name='User Type'),
	row=1, col=1
)

# Bike type pie chart
bike_counts = df['rideable_type'].value_counts()
fig.add_trace(
	go.Pie(labels=bike_counts.index, values=bike_counts.values, name='Bike Type'),
	row=1, col=2
)

fig.update_layout(height=400, title_text='Trip Characteristics')
fig.show()

In [ ]:
# Trip duration distribution
fig = px.histogram(
	df,
	x='trip_duration_minutes',
	nbins=50,
	title='Trip Duration Distribution (1-180 minutes)',
	labels={'trip_duration_minutes': 'Trip Duration (minutes)', 'count': 'Number of Trips'},
	color_discrete_sequence=['#636EFA']
)

# Add median line
median_duration = df['trip_duration_minutes'].median()
fig.add_vline(
	x=median_duration,
	line_dash='dash',
	line_color='red',
	annotation_text=f'Median: {median_duration:.1f} min'
)

fig.update_layout(height=400)
fig.show()

---

## 3. Hourly Patterns Analysis

Understanding when bikes are used throughout the day.

In [ ]:
# Aggregate trips by hour
hourly_trips = df.groupby('hour_of_day').size().reset_index(name='trip_count')

# Create bar chart
fig = px.bar(
	hourly_trips,
	x='hour_of_day',
	y='trip_count',
	title='Total Trips by Hour of Day',
	labels={'hour_of_day': 'Hour of Day', 'trip_count': 'Number of Trips'},
	color_discrete_sequence=['#636EFA']
)

fig.update_layout(
	height=400,
	xaxis=dict(tickmode='linear', tick0=0, dtick=1)
)
fig.show()

# Identify peak hours
peak_hour = hourly_trips.loc[hourly_trips['trip_count'].idxmax()]
print(f"\nPeak hour: {int(peak_hour['hour_of_day'])}:00 with {int(peak_hour['trip_count']):,} trips")

In [ ]:
# Heatmap: Day of week × Hour of day
day_hour_pivot = df.groupby(['day_name', 'hour_of_day']).size().reset_index(name='trip_count')

# Create ordered day names
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_hour_pivot['day_name'] = pd.Categorical(day_hour_pivot['day_name'], categories=day_order, ordered=True)
day_hour_pivot = day_hour_pivot.sort_values('day_name')

# Pivot for heatmap
heatmap_data = day_hour_pivot.pivot(index='day_name', columns='hour_of_day', values='trip_count')

fig = px.imshow(
	heatmap_data,
	labels=dict(x='Hour of Day', y='Day of Week', color='Trips'),
	x=heatmap_data.columns,
	y=heatmap_data.index,
	color_continuous_scale='Blues',
	aspect='auto',
	title='Trip Activity Heatmap: Day of Week × Hour of Day'
)

fig.update_layout(height=500)
fig.show()

In [ ]:
# Compare member vs casual hourly patterns
hourly_by_type = df.groupby(['hour_of_day', 'member_casual']).size().reset_index(name='trip_count')

fig = px.line(
	hourly_by_type,
	x='hour_of_day',
	y='trip_count',
	color='member_casual',
	title='Hourly Usage Patterns: Member vs Casual Users',
	labels={'hour_of_day': 'Hour of Day', 'trip_count': 'Number of Trips', 'member_casual': 'User Type'},
	markers=True
)

fig.update_layout(
	height=400,
	hovermode='x unified',
	xaxis=dict(tickmode='linear', tick0=0, dtick=2)
)
fig.show()

In [ ]:
# Time period distribution
time_period_counts = df['time_period'].value_counts().reindex(
	['Morning Rush', 'Midday', 'Evening Rush', 'Night']
)

fig = px.bar(
	x=time_period_counts.index,
	y=time_period_counts.values,
	title='Trips by Time Period',
	labels={'x': 'Time Period', 'y': 'Number of Trips'},
	color_discrete_sequence=['#636EFA']
)

fig.update_layout(height=400)
fig.show()

print("\n=== TIME PERIOD DISTRIBUTION ===")
for period in ['Morning Rush', 'Midday', 'Evening Rush', 'Night']:
	count = time_period_counts[period]
	pct = count / len(df) * 100
	print(f"{period}: {count:,} trips ({pct:.1f}%)")

---

## 4. Daily and Weekly Patterns

Understanding weekday vs weekend usage differences.

In [ ]:
# Trips by day of week
day_counts = df['day_name'].value_counts().reindex(
	['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

fig = px.bar(
	x=day_counts.index,
	y=day_counts.values,
	title='Total Trips by Day of Week',
	labels={'x': 'Day of Week', 'y': 'Number of Trips'},
	color_discrete_sequence=['#636EFA']
)

fig.update_layout(height=400)
fig.show()

# Calculate daily average
unique_days = df.groupby(['day_name', 'date']).size().groupby('day_name').size()
avg_trips_per_day = (day_counts / unique_days).reindex(
	['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

print("\n=== AVERAGE TRIPS PER DAY ===")
for day, avg in avg_trips_per_day.items():
	print(f"{day}: {avg:.0f} trips/day")

In [ ]:
# Weekday vs Weekend comparison
weekday_weekend = df.groupby('is_weekend').size()
weekday_trips = weekday_weekend[False]
weekend_trips = weekday_weekend[True]

# Count unique weekdays vs weekend days
unique_weekdays = len(df[~df['is_weekend']]['date'].unique())
unique_weekend_days = len(df[df['is_weekend']]['date'].unique())

avg_weekday = weekday_trips / unique_weekdays
avg_weekend = weekend_trips / unique_weekend_days

comparison_df = pd.DataFrame({
	'Day Type': ['Weekday', 'Weekend'],
	'Total Trips': [weekday_trips, weekend_trips],
	'Avg Trips per Day': [avg_weekday, avg_weekend]
})

fig = make_subplots(
	rows=1, cols=2,
	subplot_titles=('Total Trips', 'Average Trips per Day')
)

fig.add_trace(
	go.Bar(x=comparison_df['Day Type'], y=comparison_df['Total Trips'], name='Total'),
	row=1, col=1
)

fig.add_trace(
	go.Bar(x=comparison_df['Day Type'], y=comparison_df['Avg Trips per Day'], name='Average'),
	row=1, col=2
)

fig.update_layout(height=400, title_text='Weekday vs Weekend Usage', showlegend=False)
fig.show()

print("\n=== WEEKDAY VS WEEKEND COMPARISON ===")
print(f"Total weekday trips: {weekday_trips:,} ({weekday_trips/len(df)*100:.1f}%)")
print(f"Total weekend trips: {weekend_trips:,} ({weekend_trips/len(df)*100:.1f}%)")
print(f"\nAverage trips per weekday: {avg_weekday:.0f}")
print(f"Average trips per weekend day: {avg_weekend:.0f}")
print(f"\nWeekday days are {(avg_weekday/avg_weekend - 1)*100:+.1f}% {'higher' if avg_weekday > avg_weekend else 'lower'} than weekend days")

In [ ]:
# Member vs casual by day type
day_type_user = df.groupby(['is_weekend', 'member_casual']).size().reset_index(name='trip_count')
day_type_user['day_type'] = day_type_user['is_weekend'].map({False: 'Weekday', True: 'Weekend'})

fig = px.bar(
	day_type_user,
	x='day_type',
	y='trip_count',
	color='member_casual',
	title='User Type Distribution: Weekday vs Weekend',
	labels={'day_type': 'Day Type', 'trip_count': 'Number of Trips', 'member_casual': 'User Type'},
	barmode='group'
)

fig.update_layout(height=400)
fig.show()

---

## 5. Seasonal and Monthly Patterns

Understanding long-term trends and seasonal variations.

In [ ]:
# Monthly time series
monthly_trips = df.groupby('month_name').size().reset_index(name='trip_count')
monthly_trips = monthly_trips.sort_values('month_name')

fig = px.line(
	monthly_trips,
	x='month_name',
	y='trip_count',
	title='Monthly Trip Totals (Jan 2024 - Oct 2025)',
	labels={'month_name': 'Month', 'trip_count': 'Number of Trips'},
	markers=True
)

fig.update_layout(
	height=500,
	hovermode='x unified',
	xaxis=dict(
		tickangle=-45,
		tickmode='array',
		tickvals=monthly_trips['month_name'],
		ticktext=monthly_trips['month_name']
	)
)
fig.show()

# Identify peak and lowest months
peak_month = monthly_trips.loc[monthly_trips['trip_count'].idxmax()]
lowest_month = monthly_trips.loc[monthly_trips['trip_count'].idxmin()]

print(f"\nPeak month: {peak_month['month_name']} with {int(peak_month['trip_count']):,} trips")
print(f"Lowest month: {lowest_month['month_name']} with {int(lowest_month['trip_count']):,} trips")
print(f"Peak/Lowest ratio: {peak_month['trip_count'] / lowest_month['trip_count']:.2f}×")

In [ ]:
# Seasonal comparison
season_order = ['Winter', 'Spring', 'Summer', 'Fall']
seasonal_trips = df.groupby('season').size().reindex(season_order)

# Count months in each season
season_months = df.groupby('season')['month_name'].nunique().reindex(season_order)
avg_trips_per_month = seasonal_trips / season_months

fig = make_subplots(
	rows=1, cols=2,
	subplot_titles=('Total Trips by Season', 'Average Trips per Month')
)

fig.add_trace(
	go.Bar(x=seasonal_trips.index, y=seasonal_trips.values, name='Total'),
	row=1, col=1
)

fig.add_trace(
	go.Bar(x=avg_trips_per_month.index, y=avg_trips_per_month.values, name='Average'),
	row=1, col=2
)

fig.update_layout(height=400, title_text='Seasonal Patterns', showlegend=False)
fig.show()

print("\n=== SEASONAL COMPARISON ===")
for season in season_order:
	total = seasonal_trips[season]
	avg = avg_trips_per_month[season]
	pct = total / len(df) * 100
	print(f"{season}: {total:,} trips ({pct:.1f}%), avg {avg:,.0f} trips/month")

In [ ]:
# Seasonal hourly patterns
season_hour = df.groupby(['season', 'hour_of_day']).size().reset_index(name='trip_count')

fig = px.line(
	season_hour,
	x='hour_of_day',
	y='trip_count',
	color='season',
	category_orders={'season': season_order},
	title='Hourly Patterns by Season',
	labels={'hour_of_day': 'Hour of Day', 'trip_count': 'Number of Trips', 'season': 'Season'},
	markers=True
)

fig.update_layout(
	height=500,
	hovermode='x unified',
	xaxis=dict(tickmode='linear', tick0=0, dtick=2)
)
fig.show()

---

## 6. Key Findings Summary

### Answer to Research Question 1: Temporal Usage Patterns

In [ ]:
# Create summary statistics table
summary_stats = {
	'Metric': [
		'Total Trips',
		'Date Range',
		'Avg Trips per Day',
		'Peak Hour',
		'Most Active Day',
		'Weekday vs Weekend',
		'Peak Season',
		'Peak Month',
		'Seasonal Variation',
		'Member Percentage',
		'Electric Bike Percentage'
	],
	'Value': [
		f"{len(df):,}",
		f"{df['started_at'].min().date()} to {df['started_at'].max().date()}",
		f"{len(df) / total_days:.0f}",
		f"{int(peak_hour['hour_of_day'])}:00 ({int(peak_hour['trip_count']):,} trips)",
		f"{day_counts.idxmax()} ({day_counts.max():,} trips)",
		f"Weekday {avg_weekday:.0f}/day, Weekend {avg_weekend:.0f}/day",
		f"{seasonal_trips.idxmax()} ({seasonal_trips.max():,} trips)",
		f"{peak_month['month_name']} ({int(peak_month['trip_count']):,} trips)",
		f"{peak_month['trip_count'] / lowest_month['trip_count']:.2f}× between peak and lowest",
		f"{(df['member_casual'] == 'member').sum() / len(df) * 100:.1f}%",
		f"{(df['rideable_type'] == 'electric_bike').sum() / len(df) * 100:.1f}%"
	]
}

summary_df = pd.DataFrame(summary_stats)
print("\n" + "="*80)
print("TEMPORAL PATTERNS - KEY FINDINGS SUMMARY")
print("="*80)
print(summary_df.to_string(index=False))
print("="*80)

### Interpretation

**Time of Day Patterns:**
- Usage shows clear commute patterns with peaks during morning and evening rush hours
- Member users drive weekday commute peaks, while casual users show more midday and weekend activity

**Day of Week Patterns:**
- Weekday usage dominates, consistent with Columbia University academic/commute patterns
- Weekend usage is lower but shows different hourly patterns (later starts, more spread throughout day)

**Seasonal Patterns:**
- Strong seasonal variation with summer peak and winter low
- Likely influenced by both weather conditions and academic calendar
- Peak months show 2-3× the ridership of lowest months

**Implications for Infrastructure:**
- Peak demand periods require sufficient bike/dock capacity
- Seasonal variations suggest potential for dynamic capacity adjustments
- Member vs casual patterns may require different station configurations